In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.chains import retrieval_qa

In [1]:
# Hugging Face token
import os
api_key = os.getenv('HUGGING_FACE_API_KEY')

In [6]:
# fetch the txt sample from GitHub with an HTTP GET request
import requests
URL = "https://raw.githubusercontent.com/hwchase17/chat-your-data/master/state_of_the_union.txt"
res = requests.get(URL)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)

In [5]:
# with langchain textloader load the .txt file on documents variable
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()
type(documents)

list

In [ ]:
# this function wraps the document on phrases preserving the new line (\n) operator
import textwrap

def wrap_text_preserving_new_lines(text, width=110):
    pass
    